# SF Planning Data Inventory & Governance Project

In [81]:
import pandas as pd
#Load CSV from SF OpenData
df = pd.read_csv("C://Users//Nicho//OneDrive//Desktop//SF_Project//Housing_Pipeline.csv")
#Check the first five rows
df.head()

,Project ID,Project Name,Project Status,Construction Status,Plannning Approval Address,Zip Code,Block,Lot,Supervisor District,City Analysis Neighborhood,...,100% AMI,105% AMI,110% AMI,120% AMI,130% AMI,150% AMI,AMI Undeclared,Latitude,Longitude,Location
0,2016-081,"Block 48, Phase 1 (Blocks A, B, F, G, J)",Construction,(5) First Construction Document Issued,898 La Salle Ave,94124,NaN,NaN,10,Bayview Hunters Point,...,0,0,0,3,0,0,0,37.729001,-122.377928,"(37.72900117, -122.3779277)"
1,2024-001,2970 16th Street Permanent Supportive Housing,Pre-Construction,(2) Predevelopment Feasibility,2970 16th St,94103,NaN,NaN,9,Mission,...,0,0,0,0,0,0,0,37.765272,-122.418852,"(37.7652718, -122.4188518)"
2,2016-054,469 Eddy,Construction,(4) Site Work Permit Issued,469 Eddy St,94102,337,014A,5,Tenderloin,...,0,0,0,0,0,0,0,37.783304,-122.415227,"(37.78330387, -122.4152265)"
3,2023-008,250 Laguna Honda,Pre-Construction,(2) Predevelopment Feasibility,250 Laguna Honda Blvd,94116,2819,51,7,West of Twin Peaks,...,0,0,0,0,0,0,0,37.749863,-122.460956,"(37.74986305, -122.4609559)"
4,2021-007,4712-4720 3rd St,Construction,(4) Site Work Permit Issued,4712-4720 3rd St,94123,5311,35,10,Bayview Hunters Point,...,0,0,1,0,0,0,0,37.735443,-122.390729,"(37.7354428, -122.3907294)"


## Initial Checks

In [82]:
# Basic info
df.info() # Columns, Data types, missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 67 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Project ID                                    200 non-null    object 
 1   Project Name                                  200 non-null    object 
 2   Project Status                                200 non-null    object 
 3   Construction Status                           200 non-null    object 
 4   Plannning Approval Address                    200 non-null    object 
 5   Zip Code                                      200 non-null    int64  
 6   Block                                         160 non-null    object 
 7   Lot                                           160 non-null    object 
 8   Supervisor District                           200 non-null    int64  
 9   City Analysis Neighborhood                    200 non-null    obj

In [83]:
# Check for duplicates
print(f"Duplicate rows: {df.duplicated().sum()}")

Duplicate rows: 0


## Fix Column Names & Drop Useless Columns

In [84]:
# standardize column names (lowercase, no spaces)
df.columns = df.columns.str.lower().str.replace(" ", "_")
# Drop irrelevant columns (adjust based on your analysis)
df2 = df.drop(columns=["multiphase_or_multisite_project_name","ssp_acquisition_date","issuance_of_notice_to_proceed_date", 
                      "issuance_of_building_permit_date","issuance_of_first_construction_document_date", 
                      "estimated_construction_completion_date","project_co-sponsor", "project_owner","planning_entitlements",
                      "entitlement_approval_date","section_415_declaration","project_id","project_name","construction_status", 
                      "plannning_approval_address","block","lot","supervisor_district","city_analysis_neighborhood",
                     "estimated/actual_construction_start_date","planning__case_number","longitude","latitude","location",
                      "redevelopment_project_area"])
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 42 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   project_status                    200 non-null    object
 1   zip_code                          200 non-null    int64 
 2   planning_neighborhood             200 non-null    object
 3   lead_agency                       200 non-null    object
 4   general_housing_program           200 non-null    object
 5   specific_program_area             200 non-null    object
 6   project_type                      200 non-null    object
 7   housing_tenure                    200 non-null    object
 8   project_lead_sponsor              188 non-null    object
 9   total_project_units               200 non-null    int64 
 10  mohcd_affordable_units            200 non-null    int64 
 11  %_affordable                      200 non-null    int64 
 12  sro_units             

In [76]:
total_units = df2['total_project_units'].sum()
print(f"Total project units: {total_units}")
total_affordable_units = df2['mohcd_affordable_units'].sum()
print(f"Total affordable units: {total_affordable_units}")
print(f"Percentage of affordable units: {round((total_affordable_units/total_units)*100, 2)}")

Total project units: 27367
Total affordable units: 10136
Percentage of affordable units: 37.04


## Group Data

In [86]:
# replace null values in project lead sponser
df2['project_lead_sponsor'] = df2['project_lead_sponsor'].fillna('none_available')

In [87]:
grouped = df2.groupby([
    'zip_code',
    'project_status',
    'planning_neighborhood',
    'lead_agency',
    'general_housing_program',
    'specific_program_area',
    'project_type',
    'housing_tenure',
    'project_lead_sponsor']).agg({
    'total_project_units':'sum',
    'mohcd_affordable_units':'sum',
    '%_affordable':'mean',
    'sro_units':'sum',
    'studio_units':'sum',
    '1bd_units':'sum',
    '2bd_units':'sum',
    '3bd_units':'sum',
    '4bd_units':'sum',
    '5+_bd_units':'sum',
    'mobility_units':'sum',
    'family_units':'sum',
    'senior_units':'sum',
    'tay_units':'sum',
    'homeless_units':'sum',
    'disabled_units':'sum',
    'losp_units':'sum',
    'public_housing_replacement_units':'sum',
     '20%_ami':'sum',
    '30%_ami':'sum',
    '40%_ami':'sum',
    '50%_ami':'sum',
    '55%_ami':'sum',
    '60%_ami':'sum',
    '80%_ami':'sum',
    '90%_ami':'sum',
    '100%_ami':'sum',
    '105%_ami':'sum',
    '110%_ami':'sum',
    '120%_ami':'sum',
    '130%_ami':'sum',
    '150%_ami':'sum',
    'ami_undeclared': 'sum'
    }).reset_index()
grouped.head()

,zip_code,project_status,planning_neighborhood,lead_agency,general_housing_program,specific_program_area,project_type,housing_tenure,project_lead_sponsor,total_project_units,...,60%_ami,80%_ami,90%_ami,100%_ami,105%_ami,110%_ami,120%_ami,130%_ami,150%_ami,ami_undeclared
0,94102,Building Rehabilitation (Pre-Construction),Downtown/Civic Center,MOHCD,Inclusionary Housing,Bond Financed Inclusionary Housing,Rehabilitation,Rental,Tenderloin Neighborhood Development Corporatio...,80,...,28,45,0,0,0,0,0,0,0,7
1,94102,Building Rehabilitation (Pre-Construction),Downtown/Civic Center,MOHCD,MOHCD 100% Affordable,Multifamily Rental Program,Rehabilitation,Rental,Chinatown Community Development Center,91,...,0,0,0,0,0,0,0,0,0,0
2,94102,Building Rehabilitation (Pre-Construction),Downtown/Civic Center,MOHCD,MOHCD Preservation,Big Sites Program,Rehabilitation,Rental,Swords to Plowshares,62,...,0,62,0,0,0,0,0,0,0,0
3,94102,Building Rehabilitation (Pre-Construction),Downtown/Civic Center,MOHCD/HSH,MOHCD 100% Affordable,HSH Homekey,Rehabilitation,Rental,Five Keys School and Programs,106,...,53,0,0,0,0,0,0,0,0,0
4,94102,Construction,Downtown/Civic Center,MOHCD,Inclusionary Housing,Inclusionary Housing Program,New Construction,Ownership,"469 Eddy Street, LLC",28,...,0,0,3,0,0,0,0,0,0,0


In [78]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 42 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   zip_code                          174 non-null    int64  
 1   project_status                    174 non-null    object 
 2   planning_neighborhood             174 non-null    object 
 3   lead_agency                       174 non-null    object 
 4   general_housing_program           174 non-null    object 
 5   specific_program_area             174 non-null    object 
 6   project_type                      174 non-null    object 
 7   housing_tenure                    174 non-null    object 
 8   project_lead_sponsor              174 non-null    object 
 9   total_project_units               174 non-null    int64  
 10  mohcd_affordable_units            174 non-null    int64  
 11  %_affordable                      174 non-null    float64
 12  sro_unit

# Export Data

In [89]:
grouped.to_csv('housing_data_cleaned.csv', index = False,encoding ='utf-8-sig')

In [88]:
total_units = grouped['total_project_units'].sum()
print(f"Total project units: {total_units}")
total_affordable_units = grouped['mohcd_affordable_units'].sum()
print(f"Total affordable units: {total_affordable_units}")
print(f"Percentage of affordable units: {round((total_affordable_units/total_units)*100, 2)}")

Total project units: 27367
Total affordable units: 10136
Percentage of affordable units: 37.04
